In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [37]:
file_path = "TrainDataset2023.xls"
df = pd.read_excel(file_path)

In [38]:
X = df.drop(columns=["pCR (outcome)", 'RelapseFreeSurvival (outcome)',"ID"])
y = df['RelapseFreeSurvival (outcome)']

In [39]:
# i used MinMaxScaler to normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
X_normalized_df = pd.DataFrame(X_normalized, columns=X.columns)
X_normalized_df['ID'] = df["ID"]

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized_df, y, test_size=0.3, random_state=42)

In [41]:
# the ANN model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1] -1,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output layer for regression, no activation function
])

In [42]:
# then i compiled the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [45]:
# train
model.fit(X_train.loc[:, X_train.columns != 'ID'], y_train, epochs=50, batch_size=32, validation_split=0.3)

Epoch 1/50
7/7 [==============================] - 1s 23ms/step - loss: 57.1149 - val_loss: 50.6185
Epoch 2/50
7/7 [==============================] - 0s 6ms/step - loss: 55.8612 - val_loss: 49.5011
Epoch 3/50
7/7 [==============================] - 0s 7ms/step - loss: 54.6611 - val_loss: 48.3808
Epoch 4/50
7/7 [==============================] - 0s 7ms/step - loss: 53.2699 - val_loss: 46.9664
Epoch 5/50
7/7 [==============================] - 0s 7ms/step - loss: 51.5437 - val_loss: 45.1915
Epoch 6/50
7/7 [==============================] - 0s 6ms/step - loss: 49.4123 - val_loss: 43.1466
Epoch 7/50
7/7 [==============================] - 0s 6ms/step - loss: 46.5914 - val_loss: 40.6439
Epoch 8/50
7/7 [==============================] - 0s 6ms/step - loss: 43.4902 - val_loss: 38.0472
Epoch 9/50
7/7 [==============================] - 0s 7ms/step - loss: 40.1755 - val_loss: 35.1685
Epoch 10/50
7/7 [==============================] - 0s 7ms/step - loss: 36.9329 - val_loss: 32.5579
Epoch 11/50
7/7 [=

In [50]:
# evaluated the model on the test set
mae = model.evaluate(X_test.loc[:, X_test.columns != 'ID']
, y_test)
print(f'Mean Absolute Error (MAE): {mae}')

# predictions
y_pred = model.predict(X_test.loc[:, X_test.columns != 'ID'])

4/4 [==============================] - 0s 2ms/step - loss: 24.6645
Mean Absolute Error (MAE): 24.664546966552734
4/4 [==============================] - 0s 2ms/step


In [52]:
y_pred.shape

(120, 1)

In [54]:
data = { 'ID': X_test["ID"], 'Prediction': y_pred.reshape((y_pred.shape[0],))}
final_df = pd.DataFrame(data)

In [55]:
final_df

,ID,Prediction
209,TRG002654,70.506615
280,TRG002763,55.779705
33,TRG002359,45.962505
210,TRG002656,62.662991
93,TRG002467,68.627609
...,...,...
60,TRG002407,22.932665
79,TRG002448,45.948597
285,TRG002771,62.066917
305,TRG002798,46.473110
